In [1]:
!pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import tensorflow as tf  # TensorFlow library for deep learning
# Importing various pre-trained models from Keras applications
from tensorflow.keras.applications import (VGG16, Xception, InceptionV3, ResNet50,
                                           EfficientNetB0, EfficientNetB1, EfficientNetB2,
                                           EfficientNetB3, EfficientNetB4, EfficientNetB5,
                                           EfficientNetB6, EfficientNetB7)
# Importing necessary modules from TensorFlow and Keras
from tensorflow.keras import layers, regularizers  # Layers and regularization techniques
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Training callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Image data preprocessing
# Importing other essential libraries for data handling and evaluation
import numpy as np  # Numerical operations library
import os  # Operating system related functionality
from glob import glob  # File searching and pattern matching
import shutil  # High-level file operations
import zipfile  # ZIP file operations
import random  # Random number generation
import pandas as pd  # Data manipulation library
import matplotlib.pyplot as plt  # Visualization library
from tensorflow.keras.callbacks import Callback  # Base class for Keras callbacks


In [ ]:
compressed_paths = glob("/content/drive/MyDrive/Elsafty_RBCs_Cellular_Images_and_Masks/Segmented images/*.zip")

In [ ]:
Save_model_at          = "/content/drive/MyDrive"  # Directory to save the trained model
Model_name             = "Classifier"  # Name for the model
Transfer_Larning_Model = EfficientNetB0  # Transfer learning model choice (EfficientNetB0, VGG16, etc.)

In [ ]:
Shuffle_Before_Split   = "no"  # Flag indicating whether to shuffle data before split ("no" or "yes")

selected_split_part    = 1  # Selected part of the images to be used for testing subset
num_split_parts        = 6  # Total number of parts of the images for dataset splitting
Give_data_balance      = "no"  # Flag indicating whether to balance data ("no" or "yes")
batch_size             = 32  # Batch size for training
Epochs_number          = 20  # Number of epochs for training
initial_LearningRate   = 0.000004  # Initial learning rate for the optimizer
Patience               = 3  # Number of epochs with no improvement after which learning rate will be reduced
min_delta              = 0.01  # Minimum change in monitored quantity for reducing the learning rate
image_width            = 80  # Width of images
image_height           = 80  # Height of images

# Slides to be excluded in this experiment (used for Leave-One-Out experiments)
Slides_to_be_excluded = ["nothing to be excluded"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install xlsxwriter

import tensorflow as tf  # TensorFlow library for deep learning
# Importing various pre-trained models from Keras applications
from tensorflow.keras.applications import (VGG16, Xception, InceptionV3, ResNet50,
                                           EfficientNetB0, EfficientNetB1, EfficientNetB2,
                                           EfficientNetB3, EfficientNetB4, EfficientNetB5,
                                           EfficientNetB6, EfficientNetB7)
# Importing necessary modules from TensorFlow and Keras
from tensorflow.keras import layers, regularizers  # Layers and regularization techniques
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Training callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Image data preprocessing
# Importing other essential libraries for data handling and evaluation
import numpy as np  # Numerical operations library
import os  # Operating system related functionality
from glob import glob  # File searching and pattern matching
import shutil  # High-level file operations
import zipfile  # ZIP file operations
import random  # Random number generation
import pandas as pd  # Data manipulation library
import matplotlib.pyplot as plt  # Visualization library
from google.colab import files  # Files module from Google Colab
from tensorflow.keras.callbacks import Callback  # Base class for Keras callbacks

########################################################################################################
##########
# Set initial configuration values

# Get paths to ZIP files containing images
compressed_paths = glob("/content/drive/MyDrive/Elsafty_RBCs_Cellular_Images_and_Masks/Segmented images/*.zip")

# Configuration parameters for model training and evaluation
Save_model_at          = "/content/drive/MyDrive"  # Directory to save the trained model
Model_name             = "Classifier"  # Name for the model
Transfer_Larning_Model = EfficientNetB0  # Transfer learning model choice (EfficientNetB0, VGG16, etc.)

### Please note that on selecting the option "Shuffle_Before_Split", the total images of each class in its folder will be randomly shuffled.
### Subsequently, these shuffled images will be divided into a number of fully separated parts.
### One-part will be allocated to testing, the second to validation, and the remaining part(s) to training.
### Therefore, no cell will be present in more than one-part, and each part will contain images from each slide/patient.
### Moreover, the shuffling will be performed with a fixed seed "order" to ensure consistency when choosing different parts for testing and validation.
### Thus, the code will split the dataset with no data mix-up or allocating images from certain slides/patients to be specific for certain subset.
### This method is useful for exploring data consistency but is not ideal for generalizing performance.
### Conversely, without shuffling, the splitting will result in better performance generalization because validation and testing will be conducted on different cases.
Shuffle_Before_Split   = "no"  # Flag indicating whether to shuffle data before split ("no" or "yes")

selected_split_part    = 1  # Selected part of the images to be used for testing subset
num_split_parts        = 6  # Total number of parts of the images for dataset splitting
Give_data_balance      = "no"  # Flag indicating whether to balance data ("no" or "yes")
batch_size             = 32  # Batch size for training
Epochs_number          = 20  # Number of epochs for training
initial_LearningRate   = 0.000004  # Initial learning rate for the optimizer
Patience               = 3  # Number of epochs with no improvement after which learning rate will be reduced
min_delta              = 0.01  # Minimum change in monitored quantity for reducing the learning rate
image_width            = 80  # Width of images
image_height           = 80  # Height of images

# Slides to be excluded in this experiment (used for Leave-One-Out experiments)
Slides_to_be_excluded = ["nothing to be excluded"]
# ["nothing to be excluded"]                                                            # Exclude nothing
# ["Slide 1 ", "Slide 5 ", "Slide 6 ", "Slide 8 ", "Slide 25"]                          #  Source 1 slides
# ["Slide 2 ", "Slide 3 ", "Slide 4 ", "Slide 7 ", "Slide 9 ", "Slide 11"]              #  Source 2 slides
# ["Slide 14", "Slide 15", "Slide 19", "Slide 20", "Slide 22", "Slide 23", "Slide 24"]  #  Source 3 slides
# ["Slide 10", "Slide 12", "Slide 13", "Slide 16", "Slide 17", "Slide 18", "Slide 21"]  #  Source 4 slides

##########
########################################################################################################

# Calculate the number of classes and extract class names from ZIP file paths
num_classes = len(compressed_paths)
classes = [path.split('/')[-1].split('.zip')[0] for path in compressed_paths]
classes = sorted(classes)

# Define directories for training, validation, test, and dataset root
training_directory = "/content/Training"
val_directory = "/content/Validation"
test_directory = "/content/Test"
root = "/content/Decompressed_dataset/"

# Display separator for clarity
print("#######################################################################")

# Display configuration information
print("Model_name = ", Model_name + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts))
print("Test_split_part = ", selected_split_part, " Total_split_parts = ", num_split_parts)
print("Shuffle_Before_Split = ", Shuffle_Before_Split)
if str(Give_data_balance).lower() != "no":
    print("Give_data_balance = ", Give_data_balance)
print("Batch Size = ", batch_size)
print("Epochs_number = ", Epochs_number)
print("initial_LearningRate = ", initial_LearningRate)
print("Patience = ", Patience, " min Delta = ", min_delta)
print("Images width = ", image_width, " Images height = ", image_height)
print("num_classes = ", num_classes)

# Attempt to remove the 'root' directory if it exists
# This step removes previously extracted data
try:
    shutil.rmtree(root[:-1])
except:
    pass

# Function to check if filename contains any of the specified slides
def contains_string(filename, strings):
    for s in strings:
        if s in filename:
            return True
    return False

# Extract files from each ZIP file
for compressed_file in compressed_paths:
    with zipfile.ZipFile(compressed_file, "r") as zip_ref:
        for file_info in zip_ref.infolist():
            if not contains_string(file_info.filename, Slides_to_be_excluded):
                zip_ref.extract(file_info, path=root)

# Remove 'training_directory', 'val_directory', and 'test_directory' if they exist
# This step clears existing directories for fresh data organization
try:
    shutil.rmtree(training_directory)
except:
    pass
try:
    shutil.rmtree(val_directory)
except:
    pass
try:
    shutil.rmtree(test_directory)
except:
    pass

# Organize images into training, validation, and test directories.
# Iterate through each class and distribute images based on the specified number of split parts.
### Please note that on selecting the option "Shuffle_Before_Split", the total images of each class in its folder will be randomly shuffled.
### Subsequently, these shuffled images will be divided into a number of fully separated parts.
### One-part will be allocated to testing, the second to validation, and the remaining part(s) to training.
### Therefore, no cell will be present in more than one-part, and each part will contain images from each slide/patient.
### Moreover, the shuffling will be performed with a fixed seed "order" to ensure consistency when choosing different parts for testing and validation.
### Thus, the code will split the dataset with no data mix-up or allocating images from certain slides/patients to be specific for certain subset.
### This method is useful for exploring data consistency but is not ideal for generalizing performance.
### Conversely, without shuffling, the splitting will result in better performance generalization because validation and testing will be conducted on different cases.
for cls in classes:
    # Create directories for each class within 'training_directory', 'val_directory', and 'test_directory'
    os.makedirs(os.path.join(training_directory + "/", cls), exist_ok=True)
    os.makedirs(os.path.join(val_directory + "/", cls), exist_ok=True)
    os.makedirs(os.path.join(test_directory + "/", cls), exist_ok=True)

    # Define the folder containing images for the current class
    class_folder = os.path.join(root, cls)

    # List all files in the class folder and shuffle their order
    allfiles = os.listdir(class_folder)

    # Shuffle images before splitting
    if str(Shuffle_Before_Split).lower() != "no":
        random.seed(1234)
        random.shuffle(allfiles)

    # Divide images into parts
    part_length = len(allfiles) // num_split_parts
    remainder = len(allfiles) % num_split_parts
    start_index = 0
    result = []

    # Create split parts with approximately equal sizes
    for _ in range(num_split_parts):
        end_index = start_index + part_length
        if remainder > 0:
            end_index += 1
            remainder -= 1
        result.append(allfiles[start_index:end_index])
        start_index = end_index

    # Determine test and validation indices based on selected part
    test_inx = selected_split_part - 1
    val_inx = selected_split_part
    if val_inx >= num_split_parts:
        val_inx = 0
    test_files = result[test_inx]
    val_files = result[val_inx]

    # Separate files for training
    new_lst = [x for x in allfiles if x not in test_files]
    train_files = [x for x in new_lst if x not in val_files]

    # Copy images to respective directories for training, validation, and test
    for file in test_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(test_directory + "/", cls, file))
    for file in val_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(val_directory + "/", cls, file))
    for file in train_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(training_directory + "/", cls, file))
    print("bring class", cls)  # Print the class name to indicate progress

# Function to count the number of files in a directory
def count_files(folder_path):
    count = 0
    for root, dirs, allfiles in os.walk(folder_path):
        count += len(allfiles)
    return count

# Calculate and print the number of files in the training directory
folder_path = training_directory
file_count = count_files(folder_path)
print("Training allfiles =", file_count)

# Calculate and print the number of files in the validation directory
folder_path = val_directory
file_count = count_files(folder_path)
print("Validation allfiles =", file_count)

# Calculate and print the number of files in the test directory
folder_path = test_directory
file_count = count_files(folder_path)
print("Test allfiles =", file_count)

# Display a separator for clarity
print("#######################################################################")

# Define the size of the images
image_size = (image_height, image_width)
# Seed for data reproducibility
seed = 123

# Define data augmentation for the training set
train_datagen = ImageDataGenerator(
    rotation_range=360,  # Rotate images up to 360 degrees
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True  # Flip images vertically
)

# Data generators for the validation, test, and full data sets are initialized without augmentation
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
fullData_datagen = ImageDataGenerator()

# Generating data batches from the directory for training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    training_directory,  # Directory containing training images
    target_size=image_size,  # Resize images to match 'image_size'
    batch_size=batch_size,  # Number of samples per gradient update
    class_mode='sparse',  # Mode for class labels (integer)
    seed=seed  # Seed for shuffling
)

validation_generator = val_datagen.flow_from_directory(
    val_directory,  # Directory containing validation images
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,  # Directory containing test images
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False  # Do not shuffle the test data during inference
)

# Use EfficientNetB0 as the base model for transfer learning
base_model = tf.keras.applications.EfficientNetB0(
    weights='imagenet',  # Initialize with pre-trained ImageNet weights
    include_top=False,  # Exclude the fully connected layers from the top
    input_shape=(image_height, image_width, 3)  # Define input image dimensions and channels
)

# Freeze the pre-trained layers to prevent updating during training
for layer in base_model.layers:
    layer.trainable = True  # Setting the layers to trainable

# Add a custom classification head on top of the pre-trained model
preprocess_input = tf.keras.applications.efficientnet.preprocess_input  # Preprocessing input function for EfficientNet
inputs = tf.keras.Input(shape=(image_height, image_width, 3))  # Define input shape
x = preprocess_input(inputs)  # Preprocess the input
x = base_model(x, training=False)  # Utilize the base model without training it
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling layer
x = tf.keras.layers.Dropout(0.2)(x)  # Dropout layer with dropout rate of 0.2
outputs = tf.keras.layers.Dense(num_classes)(x)  # Dense layer for classification
model = tf.keras.Model(inputs, outputs)  # Create the final model

# Display the model summary
print(model.summary())

# Compile the model with specified optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_LearningRate)
model.compile(
    optimizer=optimizer,
    loss=[
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Sparse categorical cross-entropy loss
    ],
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')  # Sparse categorical accuracy metric
    ]
)

# Save the model after each epoch if it performs better
save_model_after_each_epoch = tf.keras.callbacks.ModelCheckpoint(
    (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts) + '.h5'),
    verbose=1,
    save_best_only=True  # Save only the best model
)

# Define the learning rate reduction callback based on validation loss
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=Patience,  # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,
    min_delta=min_delta  # Minimum change in the monitored quantity to qualify as an improvement
)

# Compute class weights based on the training dataset distribution
class_counts = np.bincount(train_generator.classes)  # Count occurrences of each class in the training set
total_samples = sum(class_counts)  # Calculate the total number of samples
class_weights = [total_samples / (len(classes) * count) for count in class_counts]  # Compute class weights
class_weights_dict = {class_label: weight for class_label, weight in enumerate(class_weights)}  # Create a dictionary of class weights

# Check if data balancing is required and print class counts and weights if 'yes'
if str(Give_data_balance).lower() == "yes":
    print(class_counts)
    print(class_weights)

# Train the model using the configured generators and settings
steps_per_epoch = train_generator.samples // train_generator.batch_size  # Calculate steps per epoch
validation_steps = validation_generator.samples // validation_generator.batch_size  # Calculate validation steps

# Define TensorBoard
class CustomTensorBoardCallback_Accuracy(Callback):
    def __init__(self, metrics):
        super(CustomTensorBoardCallback_Accuracy, self).__init__()
        # Initializing the callback with a list of metrics to monitor
        self.metrics = metrics
        # Creating empty lists to store metric values, epochs, and learning rates
        self.metric_values = {metric: [] for metric in metrics}
        self.epochs = []
        self.learning_rates = []

    def on_train_begin(self, logs=None):
        pass  # No action needed when the training starts

    def on_epoch_begin(self, epoch, logs=None):
        # Adding learning rates at the beginning of each epoch
        self.learning_rates.append(self.model.optimizer.lr.numpy())

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.epochs.append(epoch + 1)  # Incrementing the epoch number by 1
        for metric in self.metrics:
            # Collecting metric values at the end of each epoch
            self.metric_values[metric].append(logs.get(metric))

        # Plotting metrics and learning rates after each epoch
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Creating a plot figure
        lines = []
        colors = ['blue', 'orange', 'green', 'red']  # Colors for different metrics

        # Plotting each metric curve
        for i, metric in enumerate(self.metrics):
            line, = ax1.plot(self.epochs, self.metric_values[metric], '-', color=colors[i], label=metric)
            lines.append(line)

        # Adding a line for the learning rate
        lr_line, = ax1.plot(self.epochs, self.learning_rates, '--', label='Learning Rate')

        # Setting up plot labels, title, ticks, and legend
        ax1.set_ylabel('Value')
        ax1.set_title('Epoch')
        ax1.set_xticks(np.arange(min(self.epochs), max(self.epochs) + 1, 1))
        ax1.set_ylim(0, 1.0)
        ax1.set_yticks(np.arange(0, 1.1, 0.05))
        ax1.legend(lines + [lr_line], self.metrics + ['Learning Rate'])
        ax1.grid(True)
        ax1.tick_params(axis='x', top=True, bottom=False, labeltop=True, labelbottom=False)

        # Adding a secondary y-axis for better readability
        ax2 = ax1.twinx()
        ax2.set_ylim(0, 1.0)
        ax2.set_yticks(np.arange(0, 1.1, 0.05))
        ax2.yaxis.tick_right()
        ax2.yaxis.set_label_position("right")
        ax2.set_ylabel('Value')

        # Adding text labels for curves with an offset
        offset = 0.02
        for i in range(len(self.epochs)):
            x = self.epochs[i]
            y_lr = self.learning_rates[i]
            y_val_loss = self.metric_values['val_loss'][i]
            y_val_acc = self.metric_values['val_accuracy'][i]
            lr_label = f'{y_lr:.1e}'
            lr_label = (lr_label.replace("e-0", "e-").replace("0e", "e"))[2:]
            val_loss_label = str(int(y_val_loss * 100))
            val_acc_label = str(int(y_val_acc * 100))
            ax1.text(x, y_lr - (offset * 4), lr_label, ha='center', va='bottom', color=lr_line.get_color())
            ax1.text(x, y_val_loss - (offset * 3), val_loss_label, ha='center', va='bottom', color=colors[3])
            ax1.text(x, y_val_acc + offset, val_acc_label, ha='center', va='bottom', color=colors[0])
        plt.show()

# List of metrics to monitor during training
metrics = ['val_accuracy', 'accuracy', 'loss', 'val_loss']

# Creating an instance of the CustomTensorBoardCallback_Accuracy class
custom_tensorboard_callback_accuracy = CustomTensorBoardCallback_Accuracy(metrics)

# Train the model with or without data balancing
if str(Give_data_balance).lower() == "yes":
    # Train the model with data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        class_weight=class_weights_dict,  # Apply class weights during training
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )
else:
    # Train the model without data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )

# Load the test dataset from the directory using the specified configurations
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    str(test_directory) + "/",
    labels=None,
    shuffle=False,
    image_size=image_size,
    batch_size=batch_size
)

# Make predictions on the test dataset using the trained model
rounded_predictions2 = []
for image in test_dataset:
    predictions = model.predict(image)  # Get predictions for each image in the test dataset
    rounded_predictions = np.argmax(predictions, axis=1)  # Get the class index with the highest probability
    rounded_predictions2.append(rounded_predictions)  # Append the predictions to a list
rounded_predictions2 = np.concatenate(rounded_predictions2)  # Concatenate all predictions into a single array

# Map predicted class indices to their corresponding class names
class_nameslist = []
for i in rounded_predictions2:
    class_nameslist.append(classes[i])  # Map the predicted indices to their respective class names

# Get file paths and true class labels for the test dataset
XX = test_dataset.file_paths
file_pathss = pd.DataFrame(XX, columns=['Path'])  # Create a DataFrame for file paths
unique_word_list = []
for path in XX:
    path = path.replace("/", "\\")
    unique_word = path.split("\\")[-2]  # Extract the true class label from the file path
    unique_word_list.append(unique_word)  # Append true class labels to a list
True_class_lsit = pd.DataFrame(unique_word_list, columns=['True Class'])  # Create a DataFrame for true class labels

# Create DataFrames for predicted and true class labels along with file paths
class_nameslist = pd.DataFrame(class_nameslist, columns=['Predicted Class'])
Classification_result = pd.concat([file_pathss, class_nameslist, True_class_lsit], axis=1)

# Identify misclassified samples by comparing predicted and true class labels
misclassified_result = Classification_result[
    Classification_result['Predicted Class'] != Classification_result['True Class']]

# Save misclassified samples to an Excel file
filename = (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(
    num_split_parts) + "_TestData.xlsx")
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
misclassified_result.to_excel(writer, sheet_name='Misclassified_Test_Data', index=False)  # Write misclassified data to Excel
writer.save()  # Save the Excel file

files.download(filename)  # Download the Excel file
print("************************ Test Dataset classification is completed and saved.")  # Print completion message